### LFW 데이터셋 사용

In [36]:
import argparse
import os
import numpy as np
import math
import glob

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

from sklearn.model_selection import train_test_split
from PIL import Image

## GPU setting

In [41]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPU 1 to use

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


## Hyper-parameter tuning

In [34]:

os.makedirs("images_GAN", exist_ok=True)

# parser = argparse.ArgumentParser()
parser = argparse.ArgumentParser(allow_abbrev=False)

parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=64, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=3, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=400, help="interval betwen image samples")

# opt = parser.parse_args()
opt = parser.parse_args(args=[])
print(opt)

img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False

Namespace(b1=0.5, b2=0.999, batch_size=64, channels=3, img_size=64, latent_dim=100, lr=0.0002, n_cpu=8, n_epochs=200, sample_interval=400)


## Load data

In [26]:
transformer = transforms.Compose([
    transforms.Resize(opt.img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

data = glob.glob('/home/NAS_mount2/jwkim/lfw/all_imgs/*.jpg')

# train_test_split
trainpath, testpath = train_test_split(data, test_size=0.1, shuffle=False, random_state=11)

# Customize dataset
class CustomDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        
        if self.transform:
            image = self.transform(image)

        return image
    
train_dataset = CustomDataset(trainpath, transform=transformer)
test_dataset = CustomDataset(testpath, transform=transformer)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=opt.batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=opt.batch_size, shuffle=True)


In [27]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):         # 하나의 블록(block) 정의
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))  # 배치 정규화(batch normalization) 수행(차원 동일)
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(                           # 연속적인 여러 개의 블록을 가짐 
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),         # 결과적으로 (3,64,64)인 이미지 데이터 생성 
            nn.Tanh()                                         # (-1, 1)사이의 값을 가지게 함 
        )

    def forward(self, z):                                     # nosie vertor z
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

In [30]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

In [42]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

# GPU 
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

## Training 

In [43]:
for epoch in range(opt.n_epochs):
    for i, imgs in enumerate(trainloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)       # real: 1
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)        # fake: 0

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)       # 만든 이미지가 D에게 real이미지처럼 보이게 학습 

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)           # 실제 이미지는 real이라고
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)    # 만든 이미지는 fake이라고 학습 
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(trainloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(trainloader) + i
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images_GAN/%d.png" % batches_done, nrow=5, normalize=True)

[Epoch 0/200] [Batch 0/187] [D loss: 0.715009] [G loss: 0.693844]
[Epoch 0/200] [Batch 1/187] [D loss: 0.513391] [G loss: 0.688055]
[Epoch 0/200] [Batch 2/187] [D loss: 0.477659] [G loss: 0.682845]
[Epoch 0/200] [Batch 3/187] [D loss: 0.419109] [G loss: 0.676171]
[Epoch 0/200] [Batch 4/187] [D loss: 0.401436] [G loss: 0.666117]
[Epoch 0/200] [Batch 5/187] [D loss: 0.402402] [G loss: 0.654185]
[Epoch 0/200] [Batch 6/187] [D loss: 0.394438] [G loss: 0.644260]
[Epoch 0/200] [Batch 7/187] [D loss: 0.386567] [G loss: 0.645425]
[Epoch 0/200] [Batch 8/187] [D loss: 0.393719] [G loss: 0.641853]
[Epoch 0/200] [Batch 9/187] [D loss: 0.401552] [G loss: 0.646971]
[Epoch 0/200] [Batch 10/187] [D loss: 0.390777] [G loss: 0.658088]
[Epoch 0/200] [Batch 11/187] [D loss: 0.382423] [G loss: 0.681506]
[Epoch 0/200] [Batch 12/187] [D loss: 0.380475] [G loss: 0.683620]
[Epoch 0/200] [Batch 13/187] [D loss: 0.397454] [G loss: 0.676841]
[Epoch 0/200] [Batch 14/187] [D loss: 0.388995] [G loss: 0.652009]
[Epoc